In [1]:
# Fixing imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.datasets import fetch_openml
import time

In [2]:
x, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [3]:
# Fixing ys and xs for training
# x = x.reshape(len(x), 28, 28)

real_y = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] for i in range(len(y))])
for i in range(len(y)):
    real_y[i, int(y[i])] = 1.0

In [4]:
# Splitting to train and test sets
x_train, x_test, y_train, y_test = tts(x, real_y)
x_train = x_train.reshape([-1, 28, 28, 1])
x_test = x_test.reshape([-1, 28, 28, 1])

TRAINING_BATCHES = int(len(x_train)/100)
TESTING_BATCHES = int(len(x_test)/35)

x_train_batches = np.split(x_train, TRAINING_BATCHES)
x_test_batches = np.split(x_test, TESTING_BATCHES)
y_train_batches = np.split(y_train, TRAINING_BATCHES)
y_test_batches = np.split(y_test, TESTING_BATCHES)

In [5]:
class DigitRecognizer():
    def __init__(self):
        self.x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
        self.y = tf.placeholder(tf.float32)
        
#         self.W1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
#         self.b1 = tf.Variable(tf.random_normal([32]))
#         self.W2 = tf.Variable(tf.random_normal([14*14*32, 10]))
#         self.b2 = tf.Variable(tf.random_normal([10]))
        
#         conv1 = tf.nn.bias_add(tf.nn.conv2d(self.x, self.W1, strides=[1, 1, 1, 1], padding='SAME'), self.b1)
#         pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        conv1 = tf.layers.conv2d(self.x, filters=32, kernel_size=[5, 5], strides=[1, 1], padding='SAME')
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=[2, 2], padding='SAME')
        conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=[5, 5], strides=[1, 1], padding='SAME')
        pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], strides=[2, 2], padding='SAME')
        
#         logits = tf.nn.bias_add(tf.matmul(tf.reshape(pool1, [-1, 14 * 14 * 32]), self.W2), self.b2)
    
        dense1 = tf.layers.dense(tf.layers.flatten(pool2), units=1024)
        logits = tf.layers.dense(tf.layers.flatten(dense1), units=10)
        
        self.f = tf.nn.softmax(logits)
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.f, 1), tf.argmax(self.y, 1)), tf.float32))
        self.loss = tf.losses.softmax_cross_entropy(self.y, logits)

In [6]:
model = DigitRecognizer()
mini_op = tf.train.AdamOptimizer(0.01).minimize(model.loss)

session = tf.Session()
session.run(tf.global_variables_initializer())

start = time.time()

EPOCHS = 20

for epoch in range(EPOCHS):
    start_epoch = time.time()
    print("Epoch %d" % (epoch + 1))
    for batch in range(TRAINING_BATCHES):
        session.run(mini_op, {model.x: x_train_batches[batch], model.y: y_train_batches[batch]})
    acc = session.run([model.accuracy], {model.x: x_test, model.y: y_test})
    print("Accuracy of epoch %d: %.2f%s" % (epoch, acc[0], "%"))
    end_epoch = time.time()
    print("Time for epoch %.d: %.f\n" % (epoch, (end_epoch - start_epoch)))

end = time.time()
print("Runtime: %.2f" % (end - start))

loss, accuracy = session.run([model.loss, model.accuracy], {model.x: x_test, model.y: y_test})

W0909 13:31:50.507231 140460403717952 deprecation.py:323] From <ipython-input-5-308147417fe9>:14: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0909 13:31:50.510507 140460403717952 deprecation.py:506] From /home/sindrtho/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0909 13:31:50.674664 140460403717952 deprecation.py:323] From <ipython-input-5-308147417fe9>:15: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0909 13:31:50.773081 140

Epoch 1
Accuracy of epoch 0: 0.96%
Time for epoch 0: 28

Epoch 2
Accuracy of epoch 1: 0.96%
Time for epoch 1: 30

Epoch 3
Accuracy of epoch 2: 0.97%
Time for epoch 2: 30

Epoch 4
Accuracy of epoch 3: 0.97%
Time for epoch 3: 32

Epoch 5
Accuracy of epoch 4: 0.97%
Time for epoch 4: 30

Epoch 6
Accuracy of epoch 5: 0.95%
Time for epoch 5: 30

Epoch 7
Accuracy of epoch 6: 0.94%
Time for epoch 6: 30

Epoch 8
Accuracy of epoch 7: 0.94%
Time for epoch 7: 30

Epoch 9
Accuracy of epoch 8: 0.95%
Time for epoch 8: 30

Epoch 10
Accuracy of epoch 9: 0.97%
Time for epoch 9: 30

Epoch 11
Accuracy of epoch 10: 0.97%
Time for epoch 10: 30

Epoch 12
Accuracy of epoch 11: 0.97%
Time for epoch 11: 30

Epoch 13
Accuracy of epoch 12: 0.98%
Time for epoch 12: 30

Epoch 14
Accuracy of epoch 13: 0.97%
Time for epoch 13: 30

Epoch 15
Accuracy of epoch 14: 0.97%
Time for epoch 14: 30

Epoch 16
Accuracy of epoch 15: 0.98%
Time for epoch 15: 30

Epoch 17
Accuracy of epoch 16: 0.97%
Time for epoch 16: 30

Epoch 18


In [12]:
predictions, accuracy = session.run([model.f, model.accuracy], {model.x: x_test, model.y: y_test})
for p in predictions:
    max = 0
    index = 0
    for n in range(len(p)):
        if p[n] > max:
            max = p[n]
            index = n
    for n in range(len(p)):
        p[n] = 0 if not n == index else 1
        
acc = sum([int(list(predictions[i]) == list(y_test[i])) for i in range(len(predictions))])/len(predictions)*100
print("%.2f%s accuracy" % (acc, '%'))

95.35% accuracy


In [11]:
print(accuracy)

0.9535428
